In [1]:
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis,mode
from joblib import Parallel,delayed
import zipfile
import shutil
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
seed = 100
tf.random.set_seed(seed)
np.random.seed(seed)
import os
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split,LeavePGroupsOut
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input,Activation,GRU,Bidirectional,LSTM
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import accuracy_score
import tensorflow_addons as tfa

/cerebralcortex/kessel_jupyter_virtualenv/tensorflow/lib/python3.7/site-packages/pandas-1.2.3-py3.7-linux-x86_64.egg/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def get_X_y_groups(use_standardization,n_lag):
    feature_names = ['Time of Day','Gender', 'Age', 'current_window_stress_p90', 'current_window_stress_p95', 'current_window_stress_p80',
     'current_window_stress_p5', 'current_window_stress_p10', 'current_window_stress_p20', 'current_window_stress_range_90_to_80',
     'current_window_stress_range_20_to_10', 'current_window_stress_range_90_to_10', 'current_window_stress_range_80_to_20', 'current_window_stress_median',
     'current_window_stress_iqr', 'current_window_stress_skew', 'current_window_stress_diff_p90', 'current_window_stress_diff_p95',
     'current_window_stress_diff_p80', 'current_window_stress_diff_p5', 'current_window_stress_diff_p10', 'current_window_stress_diff_p20',
     'current_window_stress_diff_range_90_to_80', 'current_window_stress_diff_range_20_to_10', 'current_window_stress_diff_range_90_to_10', 'current_window_stress_diff_range_80_to_20',
     'current_window_stress_diff_median', 'current_window_stress_diff_iqr', 'current_window_stress_diff_skew', 'time_since_last_visit_smoking_spot',
     'duration_of_stay_in_smoking_spot', 'fraction_of_minutes_spent_in_smoking_spots_out_of_observed', 'daily_stress_p90', 'daily_stress_p95',
     'daily_stress_p80', 'daily_stress_p5', 'daily_stress_p10', 'daily_stress_p20', 'daily_stress_range_90_to_80', 'daily_stress_range_20_to_10',
     'daily_stress_range_90_to_10', 'daily_stress_range_80_to_20', 'daily_stress_median', 'daily_stress_iqr', 'daily_stress_skew', 'percentage_of_stress', 'maximum_duration_of_current_stress_episode',
     'average_duration_of_current_stress_episode', 'maximum_density_of_current_stress_episode', 'average_density_of_current_stress_episode', 'average_deviation_to_daily_mean_current',
     'no_stress_till_now', 'time_since_last_stress', 'duration_of_last_stress_episode', 'average_duration_of_before_stress_episode', 'density_of_last_stress_episode',
     'average_density_of_before_stress_episode', 'deviation_to_daily_mean_of_last_stress_episode', 'percentage_of_stress_before', 'percentage_of_active_',
     'maximum_duration_of_current_activity_episode', 'average_duration_of_current_window_episode', 'no_activity_till_now', 'time_since_last_activity',
     'duration_of_last_activity_episode', 'average_duration_of_before_activity_episode', 'percentage_of_active_before', 'is_smoking', 'spread', 'distance_to_nearest_spot', 'time_spent_in_transition',
     'time_spent_in_smoking_spot']
    if use_standardization:
        data  = pickle.load(open('./data/lagged_data/obs_30_prediction_60/lagged_'+str(n_lag)+'_windows.p','rb'))
    else:
        data  = pickle.load(open('./data/lagged_data/obs_30_prediction_60/lagged_'+str(n_lag)+'_windows'+'_standardized_new'+'.p','rb'))
    n_t = data.iloc[0]['features'].shape[1]
    n_f = data.iloc[0]['features'].shape[2]
    y_time = data['time'].values
    X = np.concatenate(list(data['features']))
    n = X.shape[0]
    X1 = X
    y = data['label'].values
    y = np.int64(np.array(y))
    y[y>0] = 1
    y[y<1] = -1
    groups = data['user'].values
    X_time = data['Time of Day'].values.reshape(-1,1)
    X_gender = data.Gender.values.reshape(-1,1)
    time_oh = preprocessing.OneHotEncoder().fit(X_time)
    gender_oh = preprocessing.OneHotEncoder().fit(X_gender)
    X_time = time_oh.transform(X_time).todense()
    X_gender = gender_oh.transform(X_gender).todense()
    if use_standardization:
        X1_all,y_all,groups_all,y_time_all,X_gender_all,X_time_all = [],[],[],[],[],[]
        for g in np.unique(groups):
            X1_all.append(X1[groups==g])
            y_all.extend(list(y[groups==g]))
            groups_all.extend(list(groups[groups==g]))
            y_time_all.extend(list(y_time[groups==g]))
            X_gender_all.append(preprocessing.StandardScaler().fit_transform(X_gender[groups==g]))
            X_time_all.append(preprocessing.StandardScaler().fit_transform(X_time[groups==g]))
        X1,y,groups,y_time,X_gender,X_time =  np.concatenate(X1_all),np.array(y_all),np.array(groups_all), \
        np.array(y_time_all),np.concatenate(X_gender_all),np.concatenate(X_time_all)

    X_time = np.concatenate([np.expand_dims(X_time,axis=1)]*n_t,axis=1)
    X_gender = np.concatenate([np.expand_dims(X_gender,axis=1)]*n_t,axis=1)
    feature_names_temp = feature_names[2:]
    time_feature_columns = [feature_names[0]+'=='+a for a in time_oh.categories_[0]]
    gender_feature_columns = [feature_names[1]+'=='+a for a in gender_oh.categories_[0]]
    feature_names_final = time_feature_columns+gender_feature_columns+feature_names_temp
    X = np.concatenate([X1,X_time,X_gender],axis=2)
    return X,y,groups,feature_names_final

def split_train_test_groups(X,y,groups,n_groups=10):
    gkf = LeavePGroupsOut(n_groups=n_groups)
    for train_index,test_index in gkf.split(X,y,groups=groups):
        X_train,X_test = X[train_index],X[test_index]
        y_train,y_test = y[train_index],y[test_index]
        groups_train,groups_test = groups[train_index],groups[test_index]
        break
    return X_train,X_test,y_train,y_test,groups_train,groups_test

def get_model(X_train):
    n_t,n_f = X_train.shape[1],X_train.shape[2]
    model = Sequential()
    model.add(Input(shape=(n_t,n_f)))
#     model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(Conv1D(64,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(LSTM(80,activation='tanh',return_sequences=True))
#     model.add(Bidirectional(LSTM(20,activation='relu',return_sequences=True)))
    model.add(LSTM(10,activation='tanh',return_sequences=False))
    model.add(Dropout(.1))
    model.add(Flatten())
    model.add(Dense(10,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss=tfa.losses.focal_loss.SigmoidFocalCrossEntropy(),metrics=['acc',tf.metrics.Recall()])
    return model


In [3]:
use_standardization = True
n_lag = 10
n_groups = 20
X,y,groups,feature_names = get_X_y_groups(use_standardization,n_lag)
y[y<0] = 0
y = np.float32(y)
X_train,X_test,y_train,y_test,groups_train,groups_test = split_train_test_groups(X,y,groups,n_groups=n_groups)

In [4]:
model = get_model(X_train)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 80)            53440     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                3640      
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                110       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 57,201
Trainable params: 57,201
Non-trainable params: 0
____________________________________________________

In [6]:
import uuid
filepath = './models/'+'-'.join([str(n_lag),str(n_groups)])+'-'+str(uuid.uuid4())+'.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_recall', verbose=1, save_best_only=True, mode='max',save_weights_only=False)
es = EarlyStopping(monitor='val_recall', mode='max', verbose=0,patience=40)
callbacks_list = [es,checkpoint]
train_x,val_x,train_y,val_y = train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=100, batch_size=100,verbose=1,callbacks=callbacks_list,shuffle=True)

Epoch 1/100
428/428 [==============================] - 13s 19ms/step - loss: 0.0184 - acc: 0.9624 - recall: 0.0000e+00 - val_loss: 0.0080 - val_acc: 0.9760 - val_recall: 0.0000e+00

Epoch 00001: val_recall improved from -inf to 0.00000, saving model to ./models/10-20-3801c815-275e-434e-a586-82574c2dfe99.hdf5
Epoch 2/100
428/428 [==============================] - 7s 16ms/step - loss: 0.0084 - acc: 0.9765 - recall: 0.0057 - val_loss: 0.0073 - val_acc: 0.9776 - val_recall: 0.0700

Epoch 00002: val_recall improved from 0.00000 to 0.07004, saving model to ./models/10-20-3801c815-275e-434e-a586-82574c2dfe99.hdf5
Epoch 3/100
428/428 [==============================] - 7s 15ms/step - loss: 0.0078 - acc: 0.9771 - recall: 0.0529 - val_loss: 0.0067 - val_acc: 0.9778 - val_recall: 0.0778

Epoch 00003: val_recall improved from 0.07004 to 0.07782, saving model to ./models/10-20-3801c815-275e-434e-a586-82574c2dfe99.hdf5
Epoch 4/100
428/428 [==============================] - 7s 16ms/step - loss: 0.0070

In [7]:
model.load_weights(filepath)

In [8]:
y_pred = model.predict(X_test)

In [9]:
from sklearn.metrics import auc,roc_auc_score,f1_score
roc_auc_score(y_test,y_pred)

0.6220443503910627

In [10]:
len(np.unique(groups))

79

In [11]:
y_train

array([0., 0., 0., ..., 1., 1., 1.], dtype=float32)